In [ ]:
import warnings
import os
import tensorflow as tf
import numpy as np

warnings.filterwarnings('ignore')

In [ ]:
path = tf.keras.utils.get_file('shakespeare.txt',
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path, 'rb').read().decode(encoding='utf-8')

1122304/1115394 [==============================] - 0s 0us/step


In [ ]:
print("Length of text {} characters" .format(len(text)))
print(text[:250])


Length of text 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


In [ ]:
char2idx = {u:idx for idx, u in enumerate(vocab)}
idx2char = np.array(vocab)
textAsInt = np.array([char2idx[char] for char in text])

In [ ]:
print("{")
for char, _ in zip(char2idx, range(20)):
    print("    {:4s}: {:3d}".format(repr(char), char2idx[char]))
print("}\n....\n")

print('{}  ------> characters mapped to int -----> {}'.format(repr(text[:13]),
                                                             textAsInt[:13]))

{
    '\n':   0
    ' ' :   1
    '!' :   2
    '$' :   3
    '&' :   4
    "'" :   5
    ',' :   6
    '-' :   7
    '.' :   8
    '3' :   9
    ':' :  10
    ';' :  11
    '?' :  12
    'A' :  13
    'B' :  14
    'C' :  15
    'D' :  16
    'E' :  17
    'F' :  18
    'G' :  19
}
....

'First Citizen'  ------> characters mapped to int -----> [18 47 56 57 58  1 15 47 58 47 64 43 52]


# Readying Data

In [ ]:
seqLength = 100
examplesPerEpoch = len(text) // (seqLength + 1)

charDataset = tf.data.Dataset.from_tensor_slices(textAsInt)

In [ ]:
for i in charDataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [ ]:
sequences = charDataset.batch(seqLength + 1, drop_remainder=True)

In [ ]:
for item in sequences.take(5):
    print(repr("".join(idx2char[item.numpy()])))
    

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [ ]:
def splitInputTarget(chunk):
    inputText = chunk[:-1]
    targetText = chunk[1:]
    return inputText, targetText

In [ ]:
dataset = sequences.map(splitInputTarget)

In [ ]:
for inputEx, targetEx in dataset.take(1):
    print("Input data: ", repr("".join(idx2char[inputEx.numpy()])))
    print("Target data: ", repr("".join(idx2char[targetEx.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data:  'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [ ]:
for i, (inputIdx, targetIdx) in enumerate(zip(inputEx[:5], targetEx[:5])):
    print("step {:4d}".format(i))
    print("  input {} ({:s})".format(inputIdx, repr(idx2char[inputIdx])))
    print("  expected output {} ({:s})".format(targetIdx, repr(idx2char[targetIdx])))
    
    

step    0
  input 18 ('F')
  expected output 47 ('i')
step    1
  input 47 ('i')
  expected output 56 ('r')
step    2
  input 56 ('r')
  expected output 57 ('s')
step    3
  input 57 ('s')
  expected output 58 ('t')
step    4
  input 58 ('t')
  expected output 1 (' ')


Creating Training Sets

In [ ]:
batchSize = 64
bufSize = 10000

dataset = dataset.shuffle(bufSize).batch(batchSize, drop_remainder=True)

print(dataset)

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>


In [ ]:
vocabSize = len(vocab)
embeddingDim = 256
rnnUnits = 1024

# BUILDING MODEL

In [ ]:
def buildModel(vocabSize, embeddingDim, rnnUnits, batchSize):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocabSize, embeddingDim, batch_input_shape=[batchSize, None]),
        tf.keras.layers.GRU(rnnUnits, return_sequences=True, stateful=True,
                           recurrent_initializer="glorot_uniform"),
        tf.keras.layers.Dense(vocabSize)
    ])
    
    return model

In [ ]:
model = buildModel(vocabSize=len(vocab), embeddingDim =embeddingDim, rnnUnits=rnnUnits, batchSize=batchSize)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
print(sampled_indices)

[28 54 16  1  9 53 51  0  4 64 64 45 10 58 27 45 46 54 45 32 11 21 57 39
 46 62 46 45 23 22 63 14  5 48 35 54 38 64 23  6 57 48 13 23 22 22 61 62
  4  9 36 54 15 24 29 28 49 37 56 48 40 61 61 44 41 19 44 58 57 39 61 18
 31  8  6 59 11 44 53 13 47 23 39 16  3 42 22 43 41 48 35 15 17 37 38 17
 52 57 30 19]


In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'readiest way to make the wench amends\nIs to become her husband and her father:\nThe which will I; not'

Next Char Predictions: 
 "PpD 3om\n&zzg:tOghpgT;IsahxhgKJyB'jWpZzK,sjAKJJwx&3XpCLQPkYrjbwwfcGftsawFS.,u;foAiKaD$dJecjWCEYZEnsRG"


In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, 
                                                           logits, 
                                                           from_logits=True)

In [ ]:
example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.174753


# TRAINING MODEL

In [ ]:
model.compile(optimizer="adam", loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 100

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
172/172 [==============================] - 23s 135ms/step - loss: 2.6442
Epoch 2/100
172/172 [==============================] - 23s 134ms/step - loss: 1.9540
Epoch 3/100
172/172 [==============================] - 23s 134ms/step - loss: 1.6843
Epoch 4/100
172/172 [==============================] - 23s 134ms/step - loss: 1.5367
Epoch 5/100
172/172 [==============================] - 23s 134ms/step - loss: 1.4510
Epoch 6/100
172/172 [==============================] - 23s 134ms/step - loss: 1.3928
Epoch 7/100
172/172 [==============================] - 23s 133ms/step - loss: 1.3470
Epoch 8/100
172/172 [==============================] - 23s 133ms/step - loss: 1.3087
Epoch 9/100
172/172 [==============================] - 23s 134ms/step - loss: 1.2757
Epoch 10/100
172/172 [==============================] - 23s 134ms/step - loss: 1.2420
Epoch 11/100
172/172 [==============================] - 23s 134ms/step - loss: 1.2098
Epoch 12/100
172/172 [==============================] - 23s 134

# PREDICTION

In [ ]:
model = buildModel(vocabSize, embeddingDim, rnnUnits, batchSize=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_2 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generateText(model, startString):
    numGen = 500
    inputEval = [char2idx[s] for s in startString]
    inputEval = tf.expand_dims(inputEval, 0)
    
    textGen = []
    temperature = 1
    
    model.reset_states()
    
    for i in range(numGen):
        predictions = model(inputEval)
        predictions = tf.squeeze(predictions, 0)
        predictions /= temperature
        predictedId = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        inputEval = tf.expand_dims([predictedId], 0)
        textGen.append(idx2char[predictedId])
        
    return (startString + "".join(textGen))

In [ ]:
print(generateText(model, startString ="ROMEO: "))

ROMEO: you are not to
Cut a very eight:
If I; But thou mean'd to pure clouds,
But thou seest much shall be younger'd master. Put thy will
that lay dream'd from that my red soot an old as delail.

DUKE VINCENTIO:
And how ear in love.

PETER:
Yourself, yet famous so indeed, sons, with the
meal more than never should that Duke of her,
Even posterse onea nighty self-father.

Tell him for.

JULIET:
O, seeking council madam.

BENRY BOLINGONT:
Kind sir, news, have you been tick but off consent
If no hate him 
